In [114]:
import pandas as pd 
from sklearn.utils import shuffle
from pyvi import ViTokenizer #For split vietnamese words
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces,preprocess_string, split_alphanum, strip_short, strip_numeric
import re
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.metrics import accuracy_score
import string

In [115]:
def raw_text_preprocess(raw):
    raw = raw.lower()
    raw = re.sub(r"(http\S+)|(https\S+)", "URL_Change", raw)
    raw = re.sub(r"(www\S+)|(\S+@\S+)", "URL_Change", raw)
    raw = re.sub(r"([0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9])|([0-9][0-9]/[0-9][0-9])|([0-9][0-9]/[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = re.sub(r"([0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9])|([0-9][0-9]-[0-9][0-9])|([0-9][0-9]-[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = strip_numeric(raw)
    raw = strip_short(raw, minsize=2)
    raw = ViTokenizer.tokenize(raw)
    return raw

# Đọc mail. chuyển về list

In [116]:
doc = []
label = []
with open('data_email.txt','r', encoding= 'utf-8') as f:
    for line in f:
        doc.append(line[:])
#doc = shuffle(doc,random_state = 0)
document = []
for d in doc:
    ind = d.find('|')        
    if ind == 5:   
        label.append(1)
    elif ind == 8:
        label.append(0)
    document.append(d[ind+2:])
document = [raw_text_preprocess(d) for d in document]


# Chia 5 fold để test và train

In [117]:
def divide_fold(i):
    document_test = document[(i-1)*80:i*80]
    label_test = label[(i-1)*80:i*80]
    document_train = document[:(i-1)*80+1]+document[i*80+1:]
    label_train = label[:(i-1)*80+1]+label[i*80+1:]
    return (document_test,label_test,document_train ,label_train)

# Tạo thư viện các từ có trong dữ liệu

In [118]:
def dict_word(document):
    bow = document[0].split(" ")
    set_dict=set(bow)
    for doc in document:
        bow = doc.split(" ")
        set_dict = set_dict.union(set(bow))
  
    #set_dict = sorted(set_dict.keys())
    #print(set_dict)
    #print(len(set_dict))
    return (set_dict)
#
set_dict = dict_word(document)

# Biến mỗi mail thành một dict(word, number)

In [119]:
def read_data(document, set_dict):
    word_dict = []
    for doc in document:
        bow = doc.split(" ")
        wordDict = dict.fromkeys(set_dict, 0)
        for word in bow:
            if len(set({word}) & set_dict) ==1 :
                 wordDict[word]+=1
        word_dict.append(wordDict)
    return (word_dict)
#
word_dict = read_data(document, set_dict)
#print(word_dict[0])

# Tính TF: biến mỗi mail thành một dict(word, frequency)

In [120]:
def compute_TF(word_dict, bow):
    tf_dict = {}
    bow_count = len(bow)
    for word, count in word_dict.items():
        tf_dict[word] = count/float(bow_count)
    
    return tf_dict    
#
bow = document[0].split(" ")
tf_bowA = compute_TF(word_dict[0], bow)

# Tính IDF

In [121]:
def compute_IDF(doc_list):
    import math
    idf_dict = {}
    N = len(doc_list)
    
    #count number of documents that contain this word
    idf_dict = dict.fromkeys(doc_list[0].keys(), 0)
    for doc in doc_list:
        for word, count in doc.items():
            if count > 0:
                idf_dict[word] += 1
                
    for word, count in idf_dict.items():
        idf_dict[word] = math.log(N/float(count))
        
    return idf_dict